In [1]:
import numpy as np
import pandas as pd
import dill as dl
import matplotlib.pyplot as plt
import sys
import pickle
import itertools
import re
import os
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as sm_f

In [2]:
with open("D:/DZ/Course_6/Diploma/Data/two_time_points.csv", "rb") as inp:
    two_time_points = pickle.load(inp)

In [3]:
with open("D:/DZ/Course_6/Diploma/Data/financial_data.csv", "rb") as inp:
    temp_fin = pickle.load(inp)

In [4]:
with open("D:/DZ/Course_6/Diploma/Data/mun_data_25.csv", "rb") as inp:
    data_for_models = pd.read_csv(inp, index_col=0)

In [5]:
with open("D:/DZ/Course_6/Diploma/Data/full_ind_data.csv", "rb") as inp:
    full_ind_data = pickle.load(inp)

In [6]:
#with open("D:/DZ/Course_6/Diploma/Data/mun_data_25.csv", "rb") as inp:
#    mun_data = pd.read_csv(inp, index_col=0)

In [7]:
pd.set_option("max_columns", None)

In [8]:
two_time_points

,raion,region,treatment_1,treatment_2,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed,treatment_period
0,Владивостокский,Приморский край,1,1,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074,0
1,Владивостокский,Приморский край,1,1,2021,NaN,NaN,NaN,4.74,628623.0,NaN,0.020431,0.010109,0.003427,1
2,Хасанский район,Приморский край,0,1,2016,NaN,0.0,NaN,4.20,32139.0,NaN,0.023735,0.011735,NaN,0
3,Хасанский район,Приморский край,0,1,2021,NaN,NaN,NaN,6.20,30318.0,NaN,0.020031,0.004033,0.002294,1
4,Артемовский,Приморский край,1,1,2016,NaN,0.0,NaN,2.62,114873.0,NaN,0.009542,0.009249,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Мошенской район,Новгородская область,0,0,2021,45.00,NaN,NaN,3.32,5797.0,NaN,NaN,0.007950,NaN,1
304,Пестовский район,Новгородская область,0,1,2016,88.77,0.0,NaN,0.63,20934.0,NaN,NaN,0.003777,NaN,0
305,Пестовский район,Новгородская область,0,1,2021,75.10,NaN,NaN,0.82,19579.0,NaN,NaN,0.009602,0.014286,1
306,Хвойнинский район,Новгородская область,0,1,2016,27.61,10.0,NaN,24.65,14950.0,NaN,NaN,0.003137,0.012089,0


In [9]:
temp_fin.head()

,region,raion,sub_indicator,year,indicator,times,values,budget_type,is_amendment,amendment_date,population
0,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2013,расходы бюджета,Not present,1703.560574,actual_spending,NaN,NaN,271445.0
1,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2014,расходы бюджета,Not present,1757.246786,actual_spending,NaN,NaN,273382.0
2,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2015,расходы бюджета,Not present,1315.128876,actual_spending,NaN,NaN,276090.0
3,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2016,расходы бюджета,Not present,2508.275869,actual_spending,NaN,NaN,276691.0
4,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2017,расходы бюджета,Not present,3008.628778,actual_spending,NaN,NaN,277648.0


In [10]:
temp_fin['values'].min()

-46465.5756015087

In [11]:
full_ind_data.head()

,raion,region,treatment_1,treatment_2,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed
0,Владивостокский,Приморский край,1,1,2013,NaN,NaN,3400.00,4.05,625868.0,NaN,0.050240,0.004470,0.001211
1,Владивостокский,Приморский край,1,1,2014,NaN,NaN,3400.00,3.88,630027.0,NaN,0.020670,0.008552,0.004417
2,Владивостокский,Приморский край,1,1,2015,NaN,0.0,21529.93,5.00,631387.0,NaN,0.019753,0.009448,0.003201
3,Владивостокский,Приморский край,1,1,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074
4,Владивостокский,Приморский край,1,1,2017,NaN,0.0,NaN,3.19,633414.0,NaN,0.013158,0.003128,0.003201


In [12]:
full_ind_data = pd.merge(full_ind_data, data_for_models, on=["raion", "region"], how="left").drop(columns=["treatment_1_x", "treatment_2_x"]).rename(columns={"treatment_1_y": "treatment_1", "treatment_2_y": "treatment_2"}).reset_index(drop=True)

In [13]:
mapping = {"Приморский край": 2016, "Мурманская область": 2016, "Костромская область": 2015, "Республика Алтай": 2014, "Республика Марий Эл": 2014, "Хабаровский край": 2014, 
           "Тульская область": 2014, "Новгородская область": 2016}

# Goods

In [14]:
full_ind_data.year.max()

2021

In [15]:
for i in range(-8,4):
    if i < 0:
        string = f"minus_{abs(i)}"
    else:
        string = f'plus_{i}'
    full_ind_data[string] = 0
    for region in mapping.keys():
        full_ind_data.loc[(full_ind_data.region == region) & (full_ind_data.year == mapping[region]+5+i) & (full_ind_data.treatment_1 == 1), string] = 1

In [16]:
full_ind_data.head(10)

,raion,region,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed,treatment_1,treatment_2,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
0,Владивостокский,Приморский край,2013,NaN,NaN,3400.00,4.05,625868.0,NaN,0.050240,0.004470,0.001211,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Владивостокский,Приморский край,2014,NaN,NaN,3400.00,3.88,630027.0,NaN,0.020670,0.008552,0.004417,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Владивостокский,Приморский край,2015,NaN,0.0,21529.93,5.00,631387.0,NaN,0.019753,0.009448,0.003201,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Владивостокский,Приморский край,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Владивостокский,Приморский край,2017,NaN,0.0,NaN,3.19,633414.0,NaN,0.013158,0.003128,0.003201,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,Владивостокский,Приморский край,2018,NaN,0.0,34023.34,4.21,633102.0,NaN,0.016002,0.005096,0.002653,0,1,0,0,0,0,0,0,0,0,0,0,0,0
6,Владивостокский,Приморский край,2019,NaN,0.0,67570.22,4.48,633144.0,NaN,0.028937,0.013167,0.002075,0,1,0,0,0,0,0,0,0,0,0,0,0,0
7,Владивостокский,Приморский край,2020,NaN,NaN,106191.72,4.80,634665.0,NaN,0.025499,0.007071,0.001982,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,Владивостокский,Приморский край,2021,NaN,NaN,NaN,4.74,628623.0,NaN,0.020431,0.010109,0.003427,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,Хасанский район,Приморский край,2013,NaN,6.3,116999.00,4.30,34112.0,NaN,0.028997,0.009722,0.008197,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
treat_1_roads = full_ind_data[["raion", "region", "bad_roads", "year", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_roads.dropna(inplace=True)

roads = sm_f.ols(formula="bad_roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", 
                 data=treat_1_roads).fit()
roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              bad_roads   R-squared:                       0.866
Model:                            OLS   Adj. R-squared:                  0.844
Method:                 Least Squares   F-statistic:                     39.30
Date:                Fri, 12 May 2023   Prob (F-statistic):          2.45e-280
Time:                        23:02:42   Log-Likelihood:                -3511.0
No. Observations:                 955   AIC:                             7294.
Df Residuals:                     819   BIC:                             7955.
Df Model:                         135                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                           11.4313      3.843      2.974      0.003       3.888      18.975
C(raion)[T.Антроповский район]                                      75.1776      5.017     14.983      0.000      65.329      85.026
C(raion)[T.Аяно-Майский район]                                      88.5687     11.014      8.041      0.000      66.949     110.188
C(raion)[T.Батецкий район]                                          51.7732      5.017     10.319      0.000      41.925      61.622
C(raion)[T.Бикинский район]                                         21.2425      5.161      4.116      0.000      11.112      31.373
C(raion)[T.Боровичский район]                                       26.6088      5.625      4.731      0.000      15.568      37.649
C(raion)[T.Буйский район]                                           30.2943      5.017      6.038      0.000      20.446      40.143
C(raion)[T.Валдайский район]                                        27.7099      5.017      5.523      0.000      17.861      37.558
C(raion)[T.Ванинский район]                                         13.0312      5.161      2.525      0.012       2.901      23.162
C(raion)[T.Великий Новгород]                                        25.9176      5.017      5.166      0.000      16.069      35.766
C(raion)[T.Верхнебуреинский район]                                   9.6253      5.892      1.634      0.103      -1.940      21.190
C(raion)[T.Волжский район]                                          68.9476      5.017     13.742      0.000      59.099      78.796
C(raion)[T.Волотовский район]                                       20.5592      5.165      3.981      0.000      10.421      30.697
C(raion)[T.Вохомский район]                                         75.3443      5.017     15.017      0.000      65.496      85.193
C(raion)[T.Вяземский район]                                         -4.3801      5.017     -0.873      0.383     -14.229       5.468
C(raion)[T.Галичский район]                                         74.2043      5.017     14.789      0.000      64.356      84.053
C(raion)[T.Горномарийский район]                                    81.2154      5.017     16.187      0.000      71.367      91.064
C(raion)[T.Город Волжск]                                            19.4165      5.017      3.870      0.000       9.568      29.265
C(raion)[T.Город Горно-Алтайск]                                     12.7191      5.583      2.278      0.023       1.761      23.677
C(raion)[T.Город Козьмодемьянск]                                    55.5367      6.331      8.772      0.000      43.109      67.965
C(raion)[T.Город Комсомо

In [18]:
treat_1_roads.head()

,raion,region,bad_roads,year,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
281,Печенгский район,Мурманская область,9.00,2015,0,0,0,0,0,0,0,0,0,0,0,0
282,Печенгский район,Мурманская область,6.26,2016,0,0,0,0,0,0,0,0,0,0,0,0
283,Печенгский район,Мурманская область,7.86,2017,0,0,0,0,0,0,0,0,0,0,0,0
284,Печенгский район,Мурманская область,11.72,2018,0,0,0,0,0,0,0,0,0,0,0,0
285,Печенгский район,Мурманская область,24.80,2019,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
treat_1_schools = full_ind_data[["raion", "region", "bad_schools", "year", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_schools.dropna(inplace=True)

schools = sm_f.ols(formula="bad_schools ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_schools).fit()
schools.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            bad_schools   R-squared:                       0.452
Model:                            OLS   Adj. R-squared:                  0.314
Method:                 Least Squares   F-statistic:                     3.277
Date:                Fri, 12 May 2023   Prob (F-statistic):           2.67e-21
Time:                        23:02:44   Log-Likelihood:                -2413.9
No. Observations:                 638   AIC:                             5086.
Df Residuals:                     509   BIC:                             5661.
Df Model:                         128                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               52.1675      8.601      6.065      0.000      35.270      69.065
C(raion)[T.Анучинский район]           -51.6268      9.992     -5.167      0.000     -71.258     -31.995
C(raion)[T.Арсеньевский]               -51.6268      9.992     -5.167      0.000     -71.258     -31.995
C(raion)[T.Артемовский]                -55.8934      9.765     -5.724      0.000     -75.078     -36.709
C(raion)[T.Аяно-Майский район]         -25.4514     14.616     -1.741      0.082     -54.167       3.264
C(raion)[T.Батецкий район]             -44.9080      9.755     -4.604      0.000     -64.072     -25.744
C(raion)[T.Бикинский район]            -41.6231     11.953     -3.482      0.001     -65.106     -18.140
C(raion)[T.Боровичский район]          -44.8518     11.022     -4.069      0.000     -66.506     -23.197
C(raion)[T.Валдайский район]           -51.5747      9.755     -5.287      0.000     -70.739     -32.410
C(raion)[T.Ванинский район]            -45.0948     14.699     -3.068      0.002     -73.973     -16.217
C(raion)[T.Великий Новгород]           -47.5379      9.355     -5.081      0.000     -65.918     -29.158
C(raion)[T.Владивостокский]            -51.6268      9.992     -5.167      0.000     -71.258     -31.995
C(raion)[T.Волжский район]             -52.5157      9.355     -5.614      0.000     -70.895     -34.136
C(raion)[T.Волотовский район]          -34.9080      9.755     -3.579      0.000     -54.072     -15.744
C(raion)[T.Горномарийский район]       -50.0280      9.582     -5.221      0.000     -68.853     -31.203
C(raion)[T.Город Волжск]               -52.1528      9.456     -5.516      0.000     -70.729     -33.576
C(raion)[T.Город Горно-Алтайск]        -41.1798     10.227     -4.027      0.000     -61.272     -21.088
C(raion)[T.Город Йошкар-Ола]           -49.2153      9.456     -5.205      0.000     -67.792     -30.639
C(raion)[T.Город Козьмодемьянск]       -51.6137      9.582     -5.387      0.000     -70.439     -32.789
C(raion)[T.Город Новомосковск]         -46.2998     10.799     -4.287      0.000     -67.516     -25.083
C(raion)[T.Город Тула]                 -50.1021      9.994     -5.013      0.000     -69.737     -30.468
C(raion)[T.Дальнегорский]              -48.9468      9.992     -4.898      0.000     -68.578     -29.315
C(raion)[T.Дальнереченский]            -44.0067     10.681     -4.120      0.000     -64.992     -23.022
C(raion)[T.Демянский район]            -39.0747      9.755     -4.006      0.000     -58.239     -19.910
C(raion)[T.ЗАТО Фокино]                -52.7113     10.941     -4.818      0.000     -74.206     -31.216
C(raion)[T.Заокский район]             -10.4913     10.980     -0.955      0.340     -32.063      11.080
C(raion)[T.Звениговский район]        

In [20]:
treat_1_sewerage =  full_ind_data[["raion", "region", "year", "sewage_grid_fixed", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_sewerage.dropna(inplace=True)

sewerage = sm_f.ols(formula="sewage_grid_fixed ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_sewerage).fit()
sewerage.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      sewage_grid_fixed   R-squared:                       0.635
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                     5.425
Date:                Fri, 12 May 2023   Prob (F-statistic):           3.05e-36
Time:                        23:02:44   Log-Likelihood:                 1043.8
No. Observations:                 491   AIC:                            -1848.
Df Residuals:                     371   BIC:                            -1344.
Df Model:                         119                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                            0.0076      0.016      0.489      0.625      -0.023       0.038
C(raion)[T.Анучинский район]                                        -0.0068      0.037     -0.186      0.853      -0.079       0.065
C(raion)[T.Арсеньевский]                                            -0.0091      0.024     -0.374      0.708      -0.057       0.039
C(raion)[T.Артемовский]                                             -0.0058      0.019     -0.299      0.765      -0.044       0.032
C(raion)[T.Бикинский район]                                          0.0020      0.028      0.073      0.942      -0.053       0.057
C(raion)[T.Боровичский район]                                       -0.0082      0.022     -0.372      0.710      -0.051       0.035
C(raion)[T.Буйский район]                                           -0.0057      0.028     -0.203      0.839      -0.061       0.049
C(raion)[T.Валдайский район]                                         0.0044      0.019      0.237      0.813      -0.032       0.041
C(raion)[T.Ванинский район]                                         -0.0066      0.020     -0.336      0.737      -0.045       0.032
C(raion)[T.Великий Новгород]                                        -0.0020      0.020     -0.101      0.920      -0.040       0.036
C(raion)[T.Верхнебуреинский район]                                   0.0076      0.028      0.272      0.786      -0.047       0.063
C(raion)[T.Владивостокский]                                         -0.0057      0.019     -0.304      0.761      -0.042       0.031
C(raion)[T.Вяземский район]                                         -0.0067      0.022     -0.299      0.765      -0.051       0.037
C(raion)[T.Галичский район]                                          0.0357      0.037      0.973      0.331      -0.036       0.108
C(raion)[T.Город Горно-Алтайск]                                      0.0065      0.039      0.166      0.868      -0.071       0.084
C(raion)[T.Город Комсомольск-на-Амуре]                              -0.0055      0.020     -0.282      0.778      -0.044       0.033
C(raion)[T.Город Новомосковск]                                      -0.0069      0.029     -0.241      0.810      -0.064       0.050
C(raion)[T.Город Тула]                                              -0.0042      0.022     -0.186      0.853      -0.048       0.040
C(raion)[T.Город Хабаровск]                                      -5.859e-06      0.021     -0.000      1.000      -0.042       0.042
C(raion)[T.Дальнегорский]                                           -0.0087      0.020     -0.448      0.654      -0.047       0.030
C(raion)[T.Дальнереченск

In [21]:
treat_1_heat = full_ind_data[["raion", "region", "year", "heat_grid_fixed", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_heat.dropna(inplace=True)

heat = sm_f.ols(formula="heat_grid_fixed ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_heat).fit() 
heat.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        heat_grid_fixed   R-squared:                       0.306
Model:                            OLS   Adj. R-squared:                  0.156
Method:                 Least Squares   F-statistic:                     2.044
Date:                Fri, 12 May 2023   Prob (F-statistic):           0.000357
Time:                        23:02:45   Log-Likelihood:                 626.45
No. Observations:                 260   AIC:                            -1159.
Df Residuals:                     213   BIC:                            -991.6
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.0281      0.012      2.426      0.016       0.005       0.051
C(raion)[T.Арсеньевский]             -0.0010      0.013     -0.075      0.940      -0.028       0.026
C(raion)[T.Артемовский]              -0.0088      0.013     -0.653      0.514      -0.035       0.018
C(raion)[T.Владивостокский]           0.0037      0.013      0.271      0.786      -0.023       0.030
C(raion)[T.Дальнегорский]            -0.0071      0.013     -0.523      0.602      -0.034       0.020
C(raion)[T.Дальнереченский]           0.0155      0.017      0.897      0.370      -0.019       0.050
C(raion)[T.ЗАТО Фокино]               0.0548      0.015      3.740      0.000       0.026       0.084
C(raion)[T.Кавалеровский район]       0.0044      0.017      0.255      0.799      -0.030       0.039
C(raion)[T.Кировский район]           0.0092      0.013      0.680      0.497      -0.017       0.036
C(raion)[T.Красноармейский район]     0.0315      0.017      1.817      0.071      -0.003       0.066
C(raion)[T.Лазовский район]           0.0376      0.014      2.732      0.007       0.010       0.065
C(raion)[T.Лесозаводский]             0.0150      0.013      1.109      0.269      -0.012       0.042
C(raion)[T.Михайловский район]        0.0103      0.013      0.766      0.445      -0.016       0.037
C(raion)[T.Надеждинский район]        0.0258      0.013      1.913      0.057      -0.001       0.052
C(raion)[T.Находкинский]              0.0095      0.013      0.705      0.482      -0.017       0.036
C(raion)[T.Октябрьский район]         0.0136      0.014      0.985      0.326      -0.014       0.041
C(raion)[T.Ольгинский район]          0.0051      0.013      0.375      0.708      -0.022       0.032
C(raion)[T.Партизанский]              0.0017      0.013      0.124      0.901      -0.025       0.028
C(raion)[T.Пограничный район]         0.0342      0.015      2.344      0.020       0.005       0.063
C(raion)[T.Пожарский район]          -0.0103      0.013     -0.762      0.447      -0.037       0.016
C(raion)[T.Спасск-Дальний]            0.0084      0.017      0.486      0.628      -0.026       0.043
C(raion)[T.Спасский район]            0.0043      0.017      0.248      0.804      -0.030       0.038
C(raion)[T.Тернейский район]         -0.0069      0.014     -0.502      0.616      -0.034       0.020
C(raion)[T.Уссурийский]               0.0175      0.013      1.297      0.196      -0.009       0.044
C(raion)[T.Ханкайский район]         -0.0002      0.014     -0.014      0.989      -0.027       0.027
C(raion)[T.Хасанский район]           0.0111      0.013      0.821      0.413      -0.016       0.038
C(raion)[T.Хорольский район]          0.0175      0.014      1.273      0.204      -0.010       0.045
C(raion)[T.Черниговский

In [22]:
treat_1_water = full_ind_data[["raion", "region", "year", "water_grid_fixed", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_water.dropna(inplace=True)

water = sm_f.ols(formula="water_grid_fixed ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_water).fit()
water.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       water_grid_fixed   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.228
Method:                 Least Squares   F-statistic:                     2.729
Date:                Fri, 12 May 2023   Prob (F-statistic):           1.97e-16
Time:                        23:02:45   Log-Likelihood:                 1546.5
No. Observations:                 756   AIC:                            -2833.
Df Residuals:                     626   BIC:                            -2231.
Df Model:                         129                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                            0.0413      0.012      3.425      0.001       0.018       0.065
C(raion)[T.Антроповский район]                                      -0.0313      0.023     -1.359      0.175      -0.076       0.014
C(raion)[T.Анучинский район]                                        -0.0176      0.018     -0.969      0.333      -0.053       0.018
C(raion)[T.Арсеньевский]                                            -0.0312      0.023     -1.355      0.176      -0.076       0.014
C(raion)[T.Артемовский]                                             -0.0161      0.016     -0.992      0.321      -0.048       0.016
C(raion)[T.Батецкий район]                                          -0.0307      0.019     -1.597      0.111      -0.068       0.007
C(raion)[T.Бикинский район]                                         -0.0025      0.021     -0.122      0.903      -0.043       0.038
C(raion)[T.Боровичский район]                                       -0.0252      0.020     -1.276      0.202      -0.064       0.014
C(raion)[T.Буйский район]                                           -0.0287      0.018     -1.581      0.114      -0.064       0.007
C(raion)[T.Валдайский район]                                        -0.0195      0.016     -1.206      0.228      -0.051       0.012
C(raion)[T.Ванинский район]                                         -0.0282      0.016     -1.740      0.082      -0.060       0.004
C(raion)[T.Великий Новгород]                                        -0.0331      0.017     -1.982      0.048      -0.066      -0.000
C(raion)[T.Верхнебуреинский район]                                  -0.0138      0.019     -0.716      0.474      -0.052       0.024
C(raion)[T.Владивостокский]                                         -0.0288      0.016     -1.779      0.076      -0.061       0.003
C(raion)[T.Волотовский район]                                       -0.0083      0.019     -0.431      0.667      -0.046       0.029
C(raion)[T.Вохомский район]                                         -0.0217      0.019     -1.127      0.260      -0.059       0.016
C(raion)[T.Вяземский район]                                         -0.0212      0.017     -1.266      0.206      -0.054       0.012
C(raion)[T.Галичский район]                                         -0.0345      0.019     -1.796      0.073      -0.072       0.003
C(raion)[T.Город Горно-Алтайск]                                     -0.0245      0.020     -1.229      0.220      -0.064       0.015
C(raion)[T.Город Комсомольск-на-Амуре]                              -0.0155      0.016     -0.956      0.339      -0.047       0.016
C(raion)[T.Город Хабаров

In [23]:
table = pd.DataFrame(np.array([roads.params[(len(roads.params)-10):(len(roads.params))].values, 
                               schools.params[(len(schools.params)-10):(len(schools.params))].values, 
                               sewerage.params[(len(sewerage.params)-10):(len(sewerage.params))].values,
                               heat.params[(len(heat.params)-10):(len(heat.params))].values,
                               water.params[(len(water.params)-10):(len(water.params))].values]), columns=["-8", "-7", "-6", "-5", "-4", "-3", "-2", "0", "1", "2"])
table["Зависимая переменная"] = pd.Series(["Качество дорог", "Состояние дошкольных учреждений", "Объем ремонта канализационной сети", "Объем ремонта тепловой сети", "Объем ремонта водопроводной сети"])

In [24]:
table

,-8,-7,-6,-5,-4,-3,-2,0,1,2,Зависимая переменная
0,8.299851,10.092399,5.437883,2.982173,-1.696850,0.406445,4.278359,4.348962,4.544342,6.595776,Качество дорог
1,-7.432707,-9.103975,-6.387544,-5.790475,-8.416597,-3.205569,-6.679300,-5.953562,-26.305829,-25.784320,Состояние дошкольных учреждений
2,-0.003748,0.010756,0.004102,0.010530,0.020443,0.005068,-0.000967,-0.004238,-0.014089,-0.008600,Объем ремонта канализационной сети
3,-0.015851,-0.009686,-0.000538,-0.000445,-0.009955,0.002308,-0.017895,-0.005297,0.000000,0.000000,Объем ремонта тепловой сети
4,0.014683,-0.005165,-0.000978,-0.012889,-0.000447,-0.003083,-0.001053,-0.001444,-0.004793,-0.008980,Объем ремонта водопроводной сети


In [25]:
table.to_excel("D:/DZ/Course_6/Diploma/event_stud_25_cor.xlsx")

# Models for financial data

In [26]:
temp_fin.head()

,region,raion,sub_indicator,year,indicator,times,values,budget_type,is_amendment,amendment_date,population
0,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2013,расходы бюджета,Not present,1703.560574,actual_spending,NaN,NaN,271445.0
1,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2014,расходы бюджета,Not present,1757.246786,actual_spending,NaN,NaN,273382.0
2,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2015,расходы бюджета,Not present,1315.128876,actual_spending,NaN,NaN,276090.0
3,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2016,расходы бюджета,Not present,2508.275869,actual_spending,NaN,NaN,276691.0
4,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2017,расходы бюджета,Not present,3008.628778,actual_spending,NaN,NaN,277648.0


In [27]:
actual, initial, difference = temp_fin.loc[temp_fin.budget_type == "actual_spending"], temp_fin.loc[temp_fin.budget_type == "initial_budget"], temp_fin.loc[temp_fin.budget_type == "разница"]

In [28]:
actual = pd.pivot(actual, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()
initial = pd.pivot(initial, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()
difference = pd.pivot(difference, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()

In [29]:
for i in actual, initial, difference:
    i.rename(columns={"дорожное хозяйство (дорожные фонды)": "roads", "жилищно-коммунальное хозяйство": "housing", "образование": "education"}, inplace=True)

In [30]:
actual.head()

sub_indicator,region,raion,year,roads,housing,education
0,Костромская область,Антроповский район,2013,666.115702,1610.368144,12882.945154
1,Костромская область,Антроповский район,2014,603.232937,1922.796250,12638.114480
2,Костромская область,Антроповский район,2015,688.908710,95.255228,21901.154847
3,Костромская область,Антроповский район,2016,764.043811,203.425719,10986.123190
4,Костромская область,Антроповский район,2017,NaN,NaN,NaN


In [31]:
frames = [actual, initial, difference]

In [32]:
for j in range(len(frames)):
    #temp_fin_data = pd.DataFrame()
    #for j in mapping.keys():
    #    temp_fin_data = temp_fin_data.append(frames[i].loc[(frames[i].region == j) & ((frames[i].year == mapping[j]) | (frames[i].year == mapping[j] + 5))])
    frames[j]["treatment_period"] = [1 if i > 2018 else 0 for i in  frames[j].year]
    frames[j] = pd.merge(frames[j], data_for_models, on=["region", "raion"], how="inner").reset_index(drop=True)

In [33]:
for j in range(len(frames)): 
    for i in range(-8,4):
        if i < 0:
            string = f"minus_{abs(i)}"
        else:
            string = f'plus_{i}'
        frames[j][string] = 0
        for region in mapping.keys():
            frames[j].loc[(frames[j].region == region) & (frames[j].year == mapping[region]+5+i) & (frames[j].treatment_1 == 1), string] = 1

In [34]:
frames[0].loc[frames[0].treatment_1 == 1]

,region,raion,year,roads,housing,education,treatment_period,treatment_1,treatment_2,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
90,Костромская область,Межевской район,2013,119.932022,447.681476,13080.116533,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
91,Костромская область,Межевской район,2014,268.793039,305.110363,14721.070617,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
92,Костромская область,Межевской район,2015,366.898199,960.397961,21905.499499,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
93,Костромская область,Межевской район,2016,692.608602,752.513135,12733.019118,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
94,Костромская область,Межевской район,2017,NaN,NaN,NaN,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Хабаровский край,Город Хабаровск,2018,2366.283882,1482.057687,9182.517891,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0
1416,Хабаровский край,Город Хабаровск,2019,2333.307617,1014.289413,9728.819074,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0
1417,Хабаровский край,Город Хабаровск,2020,2418.134198,976.163275,9177.532782,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0
1418,Хабаровский край,Город Хабаровск,2021,2054.200464,1327.566750,9218.851314,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0


In [35]:
frames[2].year.max()

2022

In [36]:
#frames[0]

In [37]:
actual, initial, difference = frames

In [38]:
actual.head()

,region,raion,year,roads,housing,education,treatment_period,treatment_1,treatment_2,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
0,Костромская область,Антроповский район,2013,666.115702,1610.368144,12882.945154,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Костромская область,Антроповский район,2014,603.232937,1922.796250,12638.114480,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Костромская область,Антроповский район,2015,688.908710,95.255228,21901.154847,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Костромская область,Антроповский район,2016,764.043811,203.425719,10986.123190,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Костромская область,Антроповский район,2017,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [39]:
actual_roads_data = actual[["raion", "region", "roads", "year", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
actual_roads_data.dropna(inplace=True)
actual_roads = sm_f.ols(formula="roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=actual_roads_data).fit()
actual_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.463
Model:                            OLS   Adj. R-squared:                  0.371
Method:                 Least Squares   F-statistic:                     5.022
Date:                Fri, 12 May 2023   Prob (F-statistic):           6.24e-59
Time:                        23:03:06   Log-Likelihood:                -9488.0
No. Observations:                1147   AIC:                         1.931e+04
Df Residuals:                     978   BIC:                         2.017e+04
Df Model:                         168                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          285.7619    353.946      0.807      0.420    -408.820     980.343
C(raion)[T.Антроповский район]                                     572.0156    572.662      0.999      0.318    -551.773    1695.804
C(raion)[T.Анучинский район]                                       780.6437    517.050      1.510      0.131    -234.012    1795.299
C(raion)[T.Арсеньевский]                                           512.3630    483.348      1.060      0.289    -436.155    1460.881
C(raion)[T.Артемовский]                                            981.1595    483.348      2.030      0.043      32.641    1929.678
C(raion)[T.Аяно-Майский район]                                    2560.3153    483.348      5.297      0.000    1611.797    3508.834
C(raion)[T.Батецкий район]                                        -247.2697    483.348     -0.512      0.609   -1195.788     701.249
C(raion)[T.Бикинский район]                                       -320.8584    483.348     -0.664      0.507   -1269.377     627.660
C(raion)[T.Боровичский район]                                       99.8724    541.665      0.184      0.854    -963.087    1162.832
C(raion)[T.Буйский район]                                          894.6274    572.662      1.562      0.119    -229.161    2018.416
C(raion)[T.Валдайский район]                                       169.6356    483.348      0.351      0.726    -778.883    1118.154
C(raion)[T.Ванинский район]                                        -81.9262    483.348     -0.169      0.865   -1030.444     866.592
C(raion)[T.Великий Новгород]                                      1361.8925    483.348      2.818      0.005     413.374    2310.411
C(raion)[T.Верхнебуреинский район]                                 732.7835    483.348      1.516      0.130    -215.735    1681.302
C(raion)[T.Владивостокский]                                       2130.3599    483.348      4.408      0.000    1181.842    3078.878
C(raion)[T.Волжский район]                                          40.1653    498.390      0.081      0.936    -937.871    1018.201
C(raion)[T.Волотовский район]                                     -155.0518    498.390     -0.311      0.756   -1133.088     822.984
C(raion)[T.Вохомский район]                                       2158.2557    572.673      3.769      0.000    1034.446    3282.065
C(raion)[T.Вяземский район]                                        121.2266    483.348      0.251      0.802    -827.292    1069.745
C(raion)[T.Галичский район]                                        857.1215    540.892      1.585      0.113    -204.321    1918.564
C(raion)[T.Горномарийски

In [40]:
actual_housing_data = actual[["raion", "region", "year", "housing", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
actual_housing_data.dropna(inplace=True)
actual_housing = sm_f.ols(formula="housing ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=actual_housing_data).fit()
actual_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.884
Model:                            OLS   Adj. R-squared:                  0.864
Method:                 Least Squares   F-statistic:                     45.07
Date:                Fri, 12 May 2023   Prob (F-statistic):               0.00
Time:                        23:03:07   Log-Likelihood:                -11274.
No. Observations:                1163   AIC:                         2.289e+04
Df Residuals:                     994   BIC:                         2.374e+04
Df Model:                         168                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         2189.0684   1461.597      1.498      0.135    -679.101    5057.238
C(raion)[T.Антроповский район]                                    -357.5885   2369.835     -0.151      0.880   -5008.043    4292.866
C(raion)[T.Анучинский район]                                     -1117.4459   2139.730     -0.522      0.602   -5316.352    3081.461
C(raion)[T.Арсеньевский]                                           799.8042   2000.275      0.400      0.689   -3125.443    4725.051
C(raion)[T.Артемовский]                                            856.0682   2000.275      0.428      0.669   -3069.179    4781.315
C(raion)[T.Аяно-Майский район]                                    5.699e+04   2000.275     28.492      0.000    5.31e+04    6.09e+04
C(raion)[T.Батецкий район]                                        -442.0961   2000.275     -0.221      0.825   -4367.343    3483.151
C(raion)[T.Бикинский район]                                       1084.5129   2000.275      0.542      0.588   -2840.734    5009.760
C(raion)[T.Боровичский район]                                     -970.0272   2227.413     -0.435      0.663   -5340.998    3400.944
C(raion)[T.Буйский район]                                        -1357.0905   2554.217     -0.531      0.595   -6369.367    3655.186
C(raion)[T.Валдайский район]                                      -750.3068   2000.275     -0.375      0.708   -4675.554    3174.940
C(raion)[T.Ванинский район]                                       9769.9337   2000.275      4.884      0.000    5844.687    1.37e+04
C(raion)[T.Великий Новгород]                                       921.6896   2000.275      0.461      0.645   -3003.557    4846.936
C(raion)[T.Верхнебуреинский район]                                5032.4429   2000.275      2.516      0.012    1107.196    8957.690
C(raion)[T.Владивостокский]                                        752.1343   2000.275      0.376      0.707   -3173.113    4677.381
C(raion)[T.Волжский район]                                        -945.7444   2062.516     -0.459      0.647   -4993.130    3101.641
C(raion)[T.Волотовский район]                                      864.2418   2062.516      0.419      0.675   -3183.144    4911.627
C(raion)[T.Вохомский район]                                        401.1208   2369.866      0.169      0.866   -4249.393    5051.635
C(raion)[T.Вяземский район]                                       2856.1783   2000.275      1.428      0.154   -1069.068    6781.425
C(raion)[T.Галичский район]                                      -1150.3070   2238.382     -0.514      0.607   -5542.804    3242.190
C(raion)[T.Горномарийски

In [41]:
actual_education_data = actual[["raion", "region", "year", "education", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
actual_education_data.dropna(inplace=True)
actual_education = sm_f.ols(formula="education ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=actual_education_data).fit()
actual_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.899
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     53.39
Date:                Fri, 12 May 2023   Prob (F-statistic):               0.00
Time:                        23:03:07   Log-Likelihood:                -11053.
No. Observations:                1182   AIC:                         2.244e+04
Df Residuals:                    1013   BIC:                         2.330e+04
Df Model:                         168                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1.776e+04   1035.297     17.159      0.000    1.57e+04    1.98e+04
C(raion)[T.Антроповский район]                                   -1873.9422   1680.261     -1.115      0.265   -5171.133    1423.249
C(raion)[T.Анучинский район]                                     -3859.0112   1517.140     -2.544      0.011   -6836.108    -881.915
C(raion)[T.Арсеньевский]                                         -6579.0110   1418.262     -4.639      0.000   -9362.078   -3795.944
C(raion)[T.Артемовский]                                          -6738.1640   1418.262     -4.751      0.000   -9521.231   -3955.097
C(raion)[T.Аяно-Майский район]                                    4.413e+04   1418.262     31.116      0.000    4.13e+04    4.69e+04
C(raion)[T.Батецкий район]                                       -4923.0793   1418.262     -3.471      0.001   -7706.146   -2140.012
C(raion)[T.Бикинский район]                                       -657.5799   1418.262     -0.464      0.643   -3440.647    2125.487
C(raion)[T.Боровичский район]                                    -7315.2186   1579.130     -4.632      0.000   -1.04e+04   -4216.478
C(raion)[T.Буйский район]                                        -3105.5091   1680.261     -1.848      0.065   -6402.700     191.682
C(raion)[T.Валдайский район]                                     -7065.9048   1418.262     -4.982      0.000   -9848.972   -4282.838
C(raion)[T.Ванинский район]                                        504.7007   1418.262      0.356      0.722   -2278.366    3287.768
C(raion)[T.Великий Новгород]                                     -8048.9226   1418.262     -5.675      0.000   -1.08e+04   -5265.856
C(raion)[T.Верхнебуреинский район]                                6874.1566   1418.262      4.847      0.000    4091.090    9657.223
C(raion)[T.Владивостокский]                                      -8211.9027   1418.262     -5.790      0.000    -1.1e+04   -5428.836
C(raion)[T.Волжский район]                                       -6155.5170   1462.397     -4.209      0.000   -9025.191   -3285.843
C(raion)[T.Волотовский район]                                    -7138.7696   1462.397     -4.882      0.000      -1e+04   -4269.095
C(raion)[T.Вохомский район]                                        373.5221   1680.293      0.222      0.824   -2923.730    3670.775
C(raion)[T.Вяземский район]                                       3552.5245   1418.262      2.505      0.012     769.458    6335.591
C(raion)[T.Галичский район]                                      -3937.0045   1587.071     -2.481      0.013   -7051.327    -822.682
C(raion)[T.Горномарийски

In [42]:
initial_roads_data = initial[["raion", "region", "year", "roads", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
initial_roads_data.dropna(inplace=True)
initial_roads = sm_f.ols(formula="roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=initial_roads_data).fit()
initial_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.176
Method:                 Least Squares   F-statistic:                     2.330
Date:                Fri, 12 May 2023   Prob (F-statistic):           1.78e-13
Time:                        23:03:08   Log-Likelihood:                -7860.9
No. Observations:                 911   AIC:                         1.602e+04
Df Residuals:                     764   BIC:                         1.672e+04
Df Model:                         146                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                           80.1070    551.914      0.145      0.885   -1003.341    1163.555
C(raion)[T.Антроповский район]                                     675.1916    738.590      0.914      0.361    -774.716    2125.099
C(raion)[T.Анучинский район]                                       815.9687    843.207      0.968      0.334    -839.309    2471.247
C(raion)[T.Арсеньевский]                                           161.1737    738.590      0.218      0.827   -1288.734    1611.081
C(raion)[T.Артемовский]                                            492.8104    738.590      0.667      0.505    -957.097    1942.718
C(raion)[T.Аяно-Майский район]                                    2700.7786    738.590      3.657      0.000    1250.871    4150.686
C(raion)[T.Батецкий район]                                        -163.4711    764.939     -0.214      0.831   -1665.102    1338.160
C(raion)[T.Бикинский район]                                       -132.7269    738.590     -0.180      0.857   -1582.635    1317.181
C(raion)[T.Боровичский район]                                      506.1919    827.246      0.612      0.541   -1117.753    2130.136
C(raion)[T.Буйский район]                                          485.6950    764.939      0.635      0.526   -1015.936    1987.326
C(raion)[T.Валдайский район]                                       185.9633    764.939      0.243      0.808   -1315.668    1687.594
C(raion)[T.Ванинский район]                                        -26.5885    738.590     -0.036      0.971   -1476.496    1423.319
C(raion)[T.Великий Новгород]                                      -214.6817    738.590     -0.291      0.771   -1664.589    1235.226
C(raion)[T.Верхнебуреинский район]                                -215.3413    738.590     -0.292      0.771   -1665.249    1234.566
C(raion)[T.Владивостокский]                                       1271.0419    738.590      1.721      0.086    -178.866    2720.949
C(raion)[T.Волотовский район]                                     -165.0976    764.777     -0.216      0.829   -1666.411    1336.216
C(raion)[T.Вохомский район]                                        733.4943    764.939      0.959      0.338    -768.137    2235.126
C(raion)[T.Вяземский район]                                        109.7405    738.590      0.149      0.882   -1340.167    1559.648
C(raion)[T.Галичский район]                                        742.9047    764.939      0.971      0.332    -758.727    2244.536
C(raion)[T.Город Горно-Алтайск]                                   1768.2749    988.674      1.789      0.074    -172.564    3709.114
C(raion)[T.Город Комсомо

In [43]:
initial_housing_data = initial[["raion", "region", "year", "housing", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
initial_housing_data.dropna(inplace=True)
initial_housing = sm_f.ols(formula="housing ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=initial_housing_data).fit()
initial_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.750
Model:                            OLS   Adj. R-squared:                  0.703
Method:                 Least Squares   F-statistic:                     15.91
Date:                Fri, 12 May 2023   Prob (F-statistic):          1.14e-158
Time:                        23:03:08   Log-Likelihood:                -9363.3
No. Observations:                 929   AIC:                         1.902e+04
Df Residuals:                     781   BIC:                         1.974e+04
Df Model:                         147                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          552.0972   2325.333      0.237      0.812   -4012.545    5116.739
C(raion)[T.Антроповский район]                                   -1092.6014   3858.925     -0.283      0.777   -8667.694    6482.491
C(raion)[T.Анучинский район]                                      -428.4311   3590.298     -0.119      0.905   -7476.208    6619.346
C(raion)[T.Арсеньевский]                                           928.4250   3145.120      0.295      0.768   -5245.464    7102.315
C(raion)[T.Артемовский]                                            184.4478   3145.120      0.059      0.953   -5989.442    6358.337
C(raion)[T.Аяно-Майский район]                                    4.955e+04   3145.120     15.755      0.000    4.34e+04    5.57e+04
C(raion)[T.Батецкий район]                                        -953.1557   3256.954     -0.293      0.770   -7346.576    5440.265
C(raion)[T.Бикинский район]                                       1277.5350   3145.120      0.406      0.685   -4896.354    7451.424
C(raion)[T.Боровичский район]                                     -871.6222   3513.498     -0.248      0.804   -7768.640    6025.395
C(raion)[T.Буйский район]                                         -980.0745   3256.954     -0.301      0.764   -7373.495    5413.346
C(raion)[T.Валдайский район]                                      -947.4303   3400.061     -0.279      0.781   -7621.770    5726.909
C(raion)[T.Ванинский район]                                       9064.7576   3145.120      2.882      0.004    2890.868    1.52e+04
C(raion)[T.Великий Новгород]                                      -958.7096   3145.120     -0.305      0.761   -7132.599    5215.180
C(raion)[T.Верхнебуреинский район]                                -957.1933   3145.120     -0.304      0.761   -7131.083    5216.696
C(raion)[T.Владивостокский]                                        590.1534   3145.120      0.188      0.851   -5583.736    6764.043
C(raion)[T.Волотовский район]                                     -996.9178   3256.614     -0.306      0.760   -7389.671    5395.835
C(raion)[T.Вохомский район]                                       -778.1252   3256.954     -0.239      0.811   -7171.546    5615.295
C(raion)[T.Вяземский район]                                       2499.8620   3145.120      0.795      0.427   -3674.027    8673.751
C(raion)[T.Галичский район]                                       -819.5265   3256.954     -0.252      0.801   -7212.947    5573.894
C(raion)[T.Город Горно-Алтайск]                                   1177.9076   4199.341      0.280      0.779   -7065.425    9421.240
C(raion)[T.Город Комсомо

In [44]:
initial_education_data = initial[["raion", "region", "year", "education", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
initial_education_data.dropna(inplace=True)
initial_education = sm_f.ols(formula="education ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=initial_education_data).fit()
initial_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.942
Method:                 Least Squares   F-statistic:                     105.6
Date:                Fri, 12 May 2023   Prob (F-statistic):               0.00
Time:                        23:03:09   Log-Likelihood:                -8525.8
No. Observations:                 953   AIC:                         1.735e+04
Df Residuals:                     805   BIC:                         1.807e+04
Df Model:                         147                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1.599e+04    747.021     21.401      0.000    1.45e+04    1.75e+04
C(raion)[T.Антроповский район]                                   -6153.2268   1010.872     -6.087      0.000   -8137.484   -4168.970
C(raion)[T.Анучинский район]                                     -2230.4474   1153.921     -1.933      0.054   -4495.496      34.601
C(raion)[T.Арсеньевский]                                         -5571.2562   1010.872     -5.511      0.000   -7555.513   -3586.999
C(raion)[T.Артемовский]                                          -6552.1787   1010.872     -6.482      0.000   -8536.436   -4567.922
C(raion)[T.Аяно-Майский район]                                    3.437e+04   1010.872     34.002      0.000    3.24e+04    3.64e+04
C(raion)[T.Батецкий район]                                       -4329.1104   1046.812     -4.136      0.000   -6383.913   -2274.308
C(raion)[T.Бикинский район]                                      -1732.6383   1010.872     -1.714      0.087   -3716.895     251.619
C(raion)[T.Боровичский район]                                    -5907.0439   1127.754     -5.238      0.000   -8120.730   -3693.358
C(raion)[T.Буйский район]                                        -4855.7987   1046.812     -4.639      0.000   -6910.601   -2800.996
C(raion)[T.Валдайский район]                                     -6612.9692   1046.812     -6.317      0.000   -8667.772   -4558.167
C(raion)[T.Ванинский район]                                       2182.6928   1010.872      2.159      0.031     198.436    4166.950
C(raion)[T.Великий Новгород]                                     -1.514e+04   1010.872    -14.975      0.000   -1.71e+04   -1.32e+04
C(raion)[T.Верхнебуреинский район]                               -1.513e+04   1010.872    -14.963      0.000   -1.71e+04   -1.31e+04
C(raion)[T.Владивостокский]                                      -7449.3004   1010.872     -7.369      0.000   -9433.557   -5465.044
C(raion)[T.Волотовский район]                                    -1.501e+04   1046.709    -14.344      0.000   -1.71e+04    -1.3e+04
C(raion)[T.Вохомский район]                                      -3130.9388   1046.812     -2.991      0.003   -5185.741   -1076.136
C(raion)[T.Вяземский район]                                       4639.9189   1010.872      4.590      0.000    2655.662    6624.176
C(raion)[T.Галичский район]                                      -5011.3595   1046.812     -4.787      0.000   -7066.162   -2956.557
C(raion)[T.Город Горно-Алтайск]                                  -1832.4844   1349.655     -1.358      0.175   -4481.743     816.774
C(raion)[T.Город Комсомо

In [45]:
difference_roads_data = difference[["raion", "region", "year", "roads", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
difference_roads_data.dropna(inplace=True)
difference_roads = sm_f.ols(formula="roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=difference_roads_data).fit()
difference_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.227
Method:                 Least Squares   F-statistic:                     2.612
Date:                Fri, 12 May 2023   Prob (F-statistic):           2.13e-16
Time:                        23:03:09   Log-Likelihood:                -6454.6
No. Observations:                 798   AIC:                         1.320e+04
Df Residuals:                     652   BIC:                         1.388e+04
Df Model:                         145                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                           90.9439    326.498      0.279      0.781    -550.170     732.058
C(raion)[T.Антроповский район]                                     220.5072    534.944      0.412      0.680    -829.914    1270.929
C(raion)[T.Анучинский район]                                       416.1795    497.716      0.836      0.403    -561.141    1393.500
C(raion)[T.Арсеньевский]                                           423.2823    435.886      0.971      0.332    -432.627    1279.192
C(raion)[T.Артемовский]                                            610.1817    435.886      1.400      0.162    -245.728    1466.091
C(raion)[T.Аяно-Майский район]                                       6.9539    435.886      0.016      0.987    -848.956     862.863
C(raion)[T.Батецкий район]                                        -156.5218    451.447     -0.347      0.729   -1042.988     729.944
C(raion)[T.Бикинский район]                                       -142.2890    435.886     -0.326      0.744    -998.199     713.621
C(raion)[T.Боровичский район]                                     -103.0778    499.722     -0.206      0.837   -1084.336     878.181
C(raion)[T.Буйский район]                                          114.9926    591.678      0.194      0.846   -1046.832    1276.818
C(raion)[T.Валдайский район]                                      -146.6321    451.447     -0.325      0.745   -1033.098     739.834
C(raion)[T.Ванинский район]                                         26.8521    435.886      0.062      0.951    -829.057     882.762
C(raion)[T.Великий Новгород]                                      1628.1248    435.886      3.735      0.000     772.215    2484.034
C(raion)[T.Верхнебуреинский район]                                1079.4391    435.886      2.476      0.014     223.529    1935.349
C(raion)[T.Владивостокский]                                        957.9168    435.886      2.198      0.028     102.007    1813.826
C(raion)[T.Волотовский район]                                       -4.5683    451.369     -0.010      0.992    -890.881     881.744
C(raion)[T.Вохомский район]                                       2366.6839    591.685      4.000      0.000    1204.845    3528.523
C(raion)[T.Вяземский район]                                        -52.6785    435.886     -0.121      0.904    -908.588     803.231
C(raion)[T.Галичский район]                                        592.5228    534.877      1.108      0.268    -457.767    1642.812
C(raion)[T.Город Горно-Алтайск]                                   2057.5199    604.185      3.405      0.001     871.136    3243.904
C(raion)[T.Город Комсомо

In [46]:
difference_housing_data = difference[["raion", "region", "year", "housing", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
difference_housing_data.dropna(inplace=True)
difference_housing = sm_f.ols(formula="housing ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=difference_housing_data).fit()
difference_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.212
Method:                 Least Squares   F-statistic:                     2.514
Date:                Fri, 12 May 2023   Prob (F-statistic):           2.81e-15
Time:                        23:03:10   Log-Likelihood:                -7953.4
No. Observations:                 816   AIC:                         1.620e+04
Df Residuals:                     670   BIC:                         1.689e+04
Df Model:                         145                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1449.0042   1693.054      0.856      0.392   -1875.326    4773.334
C(raion)[T.Антроповский район]                                     704.7285   4866.212      0.145      0.885   -8850.133    1.03e+04
C(raion)[T.Анучинский район]                                      -583.4310   2606.885     -0.224      0.823   -5702.078    4535.216
C(raion)[T.Арсеньевский]                                          -158.4673   2283.202     -0.069      0.945   -4641.560    4324.625
C(raion)[T.Артемовский]                                            563.8791   2283.202      0.247      0.805   -3919.213    5046.972
C(raion)[T.Аяно-Майский район]                                    8276.4726   2283.202      3.625      0.000    3793.380    1.28e+04
C(raion)[T.Батецкий район]                                        -159.0023   2364.461     -0.067      0.946   -4801.647    4483.642
C(raion)[T.Бикинский район]                                       -240.9125   2283.202     -0.106      0.916   -4724.005    4242.180
C(raion)[T.Боровичский район]                                    -1127.6281   2603.977     -0.433      0.665   -6240.565    3985.309
C(raion)[T.Буйский район]                                         -248.7224   3621.701     -0.069      0.945   -7359.971    6862.526
C(raion)[T.Валдайский район]                                      -614.1903   2468.452     -0.249      0.804   -5461.024    4232.643
C(raion)[T.Ванинский район]                                        373.6769   2283.202      0.164      0.870   -4109.416    4856.769
C(raion)[T.Великий Новгород]                                      1823.6450   2283.202      0.799      0.425   -2659.447    6306.737
C(raion)[T.Верхнебуреинский район]                                5891.3618   2283.202      2.580      0.010    1408.269    1.04e+04
C(raion)[T.Владивостокский]                                        158.7834   2283.202      0.070      0.945   -4324.309    4641.876
C(raion)[T.Волотовский район]                                     1918.4504   2364.295      0.811      0.417   -2723.868    6560.769
C(raion)[T.Вохомский район]                                       2175.6743   3099.129      0.702      0.483   -3909.499    8260.848
C(raion)[T.Вяземский район]                                        203.1508   2283.202      0.089      0.929   -4279.942    4686.243
C(raion)[T.Галичский район]                                        102.2171   2801.492      0.036      0.971   -5398.544    5602.978
C(raion)[T.Город Горно-Алтайск]                                   -231.6299   3147.886     -0.074      0.941   -6412.538    5949.278
C(raion)[T.Город Комсомо

In [47]:
difference_education_data = difference[["raion", "region", "year", "education", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
difference_education_data.dropna(inplace=True)
difference_education = sm_f.ols(formula="education ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=difference_education_data).fit()
difference_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.529
Method:                 Least Squares   F-statistic:                     7.363
Date:                Fri, 12 May 2023   Prob (F-statistic):           6.20e-75
Time:                        23:03:11   Log-Likelihood:                -7773.8
No. Observations:                 834   AIC:                         1.584e+04
Df Residuals:                     686   BIC:                         1.654e+04
Df Model:                         147                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          887.1038   1103.714      0.804      0.422   -1279.960    3054.168
C(raion)[T.Антроповский район]                                    5052.4264   1828.730      2.763      0.006    1461.846    8643.007
C(raion)[T.Анучинский район]                                     -1518.5449   1701.485     -0.892      0.372   -4859.289    1822.199
C(raion)[T.Арсеньевский]                                         -1085.3737   1490.281     -0.728      0.467   -4011.433    1840.685
C(raion)[T.Артемовский]                                           -105.1362   1490.281     -0.071      0.944   -3031.195    2820.923
C(raion)[T.Аяно-Майский район]                                    8617.7915   1490.281      5.783      0.000    5691.733    1.15e+04
C(raion)[T.Батецкий район]                                        -162.8757   1543.299     -0.106      0.916   -3193.033    2867.282
C(raion)[T.Бикинский район]                                        492.5956   1490.281      0.331      0.741   -2433.463    3418.655
C(raion)[T.Боровичский район]                                    -1438.7750   1699.351     -0.847      0.397   -4775.329    1897.779
C(raion)[T.Буйский район]                                         4416.9747   2022.712      2.184      0.029     445.525    8388.424
C(raion)[T.Валдайский район]                                      -222.4529   1543.299     -0.144      0.885   -3252.610    2807.704
C(raion)[T.Ванинский район]                                      -2133.6686   1490.281     -1.432      0.153   -5059.727     792.390
C(raion)[T.Великий Новгород]                                      7110.4670   1490.281      4.771      0.000    4184.408       1e+04
C(raion)[T.Верхнебуреинский район]                                2.198e+04   1490.281     14.749      0.000    1.91e+04    2.49e+04
C(raion)[T.Владивостокский]                                       -766.9939   1490.281     -0.515      0.607   -3693.053    2159.065
C(raion)[T.Волотовский район]                                     7615.0230   1543.207      4.935      0.000    4585.047    1.06e+04
C(raion)[T.Вохомский район]                                       2923.8298   2022.713      1.445      0.149   -1047.622    6895.281
C(raion)[T.Вяземский район]                                      -1247.9966   1490.281     -0.837      0.403   -4174.056    1678.062
C(raion)[T.Галичский район]                                       2882.2970   1828.521      1.576      0.115    -707.873    6472.467
C(raion)[T.Город Горно-Алтайск]                                     64.6449   2054.542      0.031      0.975   -3969.302    4098.591
C(raion)[T.Город Комсомо

In [48]:
table = pd.DataFrame(np.array([actual_roads.params[(len(actual_roads.params)-10):(len(actual_roads.params))].values, 
                               actual_housing.params[(len(actual_housing.params)-10):(len(actual_housing.params))].values, 
                               actual_education.params[(len(actual_education.params)-10):(len(actual_education.params))].values,
                               initial_roads.params[(len(initial_roads.params)-10):(len(initial_roads.params))].values, 
                               initial_housing.params[(len(initial_housing.params)-10):(len(initial_housing.params))].values, 
                               initial_education.params[(len(initial_education.params)-10):(len(initial_education.params))].values,
                               difference_roads.params[(len(difference_roads.params)-10):(len(difference_roads.params))].values, 
                               difference_housing.params[(len(difference_housing.params)-10):(len(difference_housing.params))].values, 
                               difference_education.params[(len(difference_education.params)-10):(len(difference_education.params))].values]), columns=["-8", "-7", "-6", "-5", "-4", "-3", "-2", "0", "1", "2"])
table["Зависимая переменная"] = pd.Series(["Дорожное хозяйство", "ЖКХ", "Образование", "Дорожное хозяйство", "ЖКХ", "Образование", "Дорожное хозяйство", "ЖКХ", "Образование"])
table["Тип расходов"] = pd.Series(["Фактические", "Фактические", "Фактические", "Запланированные", "Запланированные", "Запланированные", "Фактические - запланированные", "Фактические - запланированные", "Фактические - запланированные"])

In [49]:
table

,-8,-7,-6,-5,-4,-3,-2,0,1,2,Зависимая переменная,Тип расходов
0,9.357532e+01,-58.580957,-189.541631,181.102213,-7.512100,-238.976895,-13.563872,-128.298899,-527.888088,-115.064716,Дорожное хозяйство,Фактические
1,6.453765e+02,831.640921,1061.099234,869.682941,585.594364,422.645128,49.171494,-132.997235,-27.754321,237.366533,ЖКХ,Фактические
2,-2.770510e+03,-1363.081770,-1137.233901,1513.451490,-408.943570,623.630495,448.529364,604.810447,653.407619,2229.409844,Образование,Фактические
3,-3.216214e-13,-263.847644,-296.489401,-188.784166,-409.819325,-464.985707,-417.509267,-601.547540,-237.126148,-794.214954,Дорожное хозяйство,Запланированные
4,1.313553e-11,3278.929888,322.015399,535.569652,214.574464,663.423761,-282.778027,-188.591324,-254.134486,521.051595,ЖКХ,Запланированные
5,-2.452890e-12,-3490.630970,-2539.973457,-1605.696146,-1282.762910,-894.500441,182.225340,-1264.483193,-1552.758422,-150.585421,Образование,Запланированные
6,-2.101897e-13,355.501298,-174.590562,-181.812789,-81.908761,-161.470608,54.958244,-164.096707,-736.748741,238.860727,Дорожное хозяйство,Фактические - запланированные
7,-1.146476e-12,-610.428257,1190.562380,1066.220598,972.557477,350.744488,819.411815,660.279657,929.877891,694.795309,ЖКХ,Фактические - запланированные
8,1.301839e-12,3060.054370,1099.409327,2383.727128,587.021679,1481.986726,-232.888074,1424.150333,2087.840533,1895.596704,Образование,Фактические - запланированные


In [50]:
table.to_excel("D:/DZ/Course_6/Diploma/event_stud_fin_25_cor.xlsx")